In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_47/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
nltk=3.5 already installed
unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


<img src='../../../common/logo_DH.png' align='left' width=35%/>

### Checkpoint: Clasificacion de texto

Trabajaremos con un dataset de noticias (en ingles) sobre diferentes temas: world, sports, business, sci/tech.

La idea es vectorizar el corpus de noticias e implementar un clasificador para identificar a qué tema pertenecen las noticias.

Trabajaremos con una versión reducida de un corpus que pueden descargar completo (~30Mb) del siguiente sitio:

https://github.com/mhjabreel/CharCnn_Keras

In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.stem.snowball import SnowballStemmer
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt

#### Ejercicio 1

Generamos el corpus:

Importen los datos con pandas. Los datos se encuentran en '../Data/ag_news_reduced.csv'. 

El dataset contiene tres columnas: la primera tiene un entero que indica a qué clase pertenece la noticia. La segunda es el título y la tercera es una descripción de la noticia.

Como no tenemos las noticias enteras sino solo un resumen, conviene que generemos el corpus concatenando el título y la descripción en un sólo texto. 

Generen una columna en el dataframe que sea la concatenación del título y la descripción. No olviden agregar un espacio en blanco para no pegar la última palabra del título con la primera de la descrición.

¿Cuántos artículos hay de cada clase?

In [3]:
df = pd.read_csv('../Data/ag_news_reduced.csv')
df.head()

,class,class_name,title,description
0,3,Business,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Business,Google IPO faces Playboy slip-up,The bidding gets underway for Google's public ...
2,3,Business,Letters,Target the abusers of legal weapons We can all...
3,3,Business,Oldsmobile: The final parking lot,Why General Motors dropped the Oldsmobile. The...
4,4,Sci/Tech,AOL to Sell Cheap PCs to Minorities and Senior...,Reuters - America Online on Thursday said it\p...


In [4]:
df['corpus']=df.apply(lambda x :x['title']+' '+x['description'], axis=1)

df.head()

,class,class_name,title,description,corpus
0,3,Business,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Wall St. Bears Claw Back Into the Black (Reute...
1,3,Business,Google IPO faces Playboy slip-up,The bidding gets underway for Google's public ...,Google IPO faces Playboy slip-up The bidding g...
2,3,Business,Letters,Target the abusers of legal weapons We can all...,Letters Target the abusers of legal weapons We...
3,3,Business,Oldsmobile: The final parking lot,Why General Motors dropped the Oldsmobile. The...,Oldsmobile: The final parking lot Why General ...
4,4,Sci/Tech,AOL to Sell Cheap PCs to Minorities and Senior...,Reuters - America Online on Thursday said it\p...,AOL to Sell Cheap PCs to Minorities and Senior...


In [5]:
df['class'].value_counts()

2    1539
4    1520
1    1486
3    1455
Name: class, dtype: int64

#### Ejercicio 2

¿Cuáles son las palabras más frecuentes dentro de cada clase?

Ayuda: Pueden vectorizar el corpus (dividido por temas) usando CountVectorizer() y luego sumar las filas de la matriz para obtener el número total de veces que aparece cada término.

Ayuda2: No olviden remover stopwords al vectorizar!

In [6]:
stopwords_en = stopwords.words('english')

vectorizer=CountVectorizer(stop_words=stopwords_en, lowercase=True, strip_accents='unicode')

In [7]:
X_1=df['corpus'][df['class']==1]
X_2=df['corpus'][df['class']==2]
X_3=df['corpus'][df['class']==3]
X_4=df['corpus'][df['class']==4]

In [8]:
X_1_vec=vectorizer.fit_transform(X_1)
X_1_vec_df=pd.DataFrame(X_1_vec.todense(), columns = vectorizer.get_feature_names())
X_1_vec_df.sum().sort_values(ascending = False)[:10]

39           471
said         362
iraq         288
ap           281
reuters      246
president    222
us           202
two          199
afp          165
minister     163
dtype: int64

In [9]:
X_2_vec=vectorizer.fit_transform(X_2)
X_2_vec_df=pd.DataFrame(X_2_vec.todense(), columns = vectorizer.get_feature_names())
X_2_vec_df.sum().sort_values(ascending = False)[:10]

39        709
ap        356
new       226
first     216
game      206
team      200
two       197
season    193
win       177
one       173
dtype: int64

In [10]:
X_3_vec=vectorizer.fit_transform(X_3)
X_3_vec_df=pd.DataFrame(X_3_vec.todense(), columns = vectorizer.get_feature_names())
X_3_vec_df.sum().sort_values(ascending = False)[:10]

39         611
reuters    422
said       341
oil        332
new        310
us         279
gt         274
lt         272
stocks     231
inc        217
dtype: int64

In [11]:
X_4_vec=vectorizer.fit_transform(X_4)
X_4_vec_df=pd.DataFrame(X_4_vec.todense(), columns = vectorizer.get_feature_names())
X_4_vec_df.sum().sort_values(ascending = False)[:10]

39           458
new          348
microsoft    287
software     211
said         204
ap           203
lt           180
gt           179
reuters      176
internet     172
dtype: int64

#### Ejercicio 3

Vectorizar el corpus usando Countvectorizer y entrenar un clasificador de tipo Multinomial Naive Bayes (MultinomialNB). Para empezar, hagan un simple train_test_split de los datos y vean la performance (accuracy) en el set de validacion.

Luego vean si pueden mejorar esa performance optimizando el parámetro alpha del modelo haciendo una Gridsearch cross validation.

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X = df.corpus
y = df.class_name

# vectorizer_tfidf = TfidfTransformer()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y,random_state = 1203)


X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV


In [15]:
mnb = MultinomialNB().fit(X_train, y_train)

skf = StratifiedKFold(shuffle = True, random_state=1203)

cvs = cross_val_score(mnb, X_train, y_train, cv=skf)

In [16]:
print('El accuracy del set de validacion es: ',cvs.mean())

El accuracy del set de validacion es:  0.8822222222222222


In [17]:
# Intentemos mejorar la performance a partir de gridsearch
param_grid = {'alpha':np.arange(0.1,1, 0.1)}


gs = GridSearchCV(mnb, param_grid, cv = skf)

In [18]:
gs.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1203, shuffle=True),
             estimator=MultinomialNB(),
             param_grid={'alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])})

In [19]:
best_mnb= gs.best_estimator_  

In [20]:
cvs = cross_val_score(best_mnb, X_train, y_train, cv=skf)
print('El accuracy del set de validacion es: ',cvs.mean())

El accuracy del set de validacion es:  0.8842222222222222


#### Ejercicio 4

Repetir lo anterior removiendo stopwords al vectorizar.

¿Cambia la performance?  ¿Cuánto se redujo la dimensionalidad?

#### Ejercicio 5

Utilizar los parámetros min_df y max_df para remover términos que aparecen en muy pocos documentos o que aparecen en demasiados (stopwords específicas del corpus).

Reentrenar el modelo y ver si cambia la performance.



In [21]:
vectorizer=CountVectorizer(stop_words=stopwords_en, 
                           lowercase=True, 
                           strip_accents='unicode', 
                           max_df=100, min_df = 1)

In [22]:
X = df.corpus
y = df.class_name

# vectorizer_tfidf = TfidfTransformer()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y,random_state = 1203)


X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [23]:
mnb = MultinomialNB()

skf = StratifiedKFold(shuffle = True, random_state=1203)


In [24]:
param_grid = {'alpha':np.arange(0.1,1, 0.1)}


gs = GridSearchCV(mnb, param_grid, cv = skf).fit(X_train, y_train)

In [25]:
best_mnb = gs.best_estimator_  
cvs = cross_val_score(best_mnb, X_train, y_train, cv=skf)
print('El accuracy del set de validacion es: ',cvs.mean())

El accuracy del set de validacion es:  0.8788888888888888


#### Ejercicio 6

Repetir el análisis incluyendo bigramas en la vectorizacion.

In [50]:
stopwords_en.append('reuters') #removemos tokenz de nuestro vocabulario que no tienen valor y estan entre las mas repetidas
stopwords_en.append('39')

vectorizer=CountVectorizer(stop_words=stopwords_en, 
                           lowercase=True, 
                           strip_accents='unicode', 
                           max_df=0.8, min_df = 2,
                           ngram_range=(1,3))# incluimos bigramas y trigramas

In [51]:
X = df.corpus
y = df.class_name

vectorizer_tfidf = TfidfVectorizer()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y,random_state = 1203)


X_train = vectorizer_tfidf.fit_transform(X_train)
X_test = vectorizer_tfidf.transform(X_test)

In [52]:
mnb = MultinomialNB()

skf = StratifiedKFold(shuffle = True, random_state=1203)


In [53]:
param_grid = {'alpha':np.arange(0.1,1, 0.1)}


gs = GridSearchCV(mnb, param_grid, cv = skf).fit(X_train, y_train)

In [54]:
best_mnb = gs.best_estimator_  
cvs = cross_val_score(best_mnb, X_train, y_train, cv=skf)
print('El accuracy del set de validacion es: ',cvs.mean())

El accuracy del set de validacion es:  0.8828888888888888


In [55]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, best_mnb.predict(X_test))

0.878